In [3]:
import MDAnalysis as mda
from pathlib import Path

In [9]:
from MDAnalysis.analysis import rms
import numpy as np
import matplotlib.pyplot as plt


def compute_trajectory_rmsd(ref_pdb, traj_file, selection="backbone", output_path=None):
    """
    Compute RMSD for each frame in a trajectory compared to a reference structure.

    Args:
        ref_pdb: Path to reference PDB file
        traj_file: Path to trajectory file (DCD, XTC, TRR, etc.)
        selection: Atom selection string (default: 'backbone')
        output_path: Path to save the RMSD plot. If None, saves as 'rmsd_plot.png'

    Returns:
        tuple: (time array, RMSD array)
    """
    # Load reference and trajectory
    u = mda.Universe(ref_pdb, traj_file)
    ref = mda.Universe(ref_pdb)

    # Select atoms for RMSD calculation
    mobile = u.select_atoms(selection)
    reference = ref.select_atoms(selection)

    # Ensure we have matching selections
    if len(mobile) != len(reference):
        raise ValueError(
            f"Selections have different lengths: {len(mobile)} vs {len(reference)}"
        )

    # Calculate RMSD
    R = rms.RMSD(
        mobile, reference, select=selection, ref_frame=0
    )  # superimpose on first frame
    R.run()

    # Get time and RMSD data
    time = np.arange(len(R.results.rmsd)) * u.trajectory.dt
    rmsd = R.results.rmsd[:, 2]  # Column 2 contains RMSD values

    # Create plot
    plt.figure(figsize=(10, 6))
    plt.plot(time, rmsd)
    plt.xlabel("Time (ps)")
    plt.ylabel("RMSD (Å)")
    plt.title(f"RMSD over time ({selection})")

    # Save plot
    if output_path is None:
        output_path = "rmsd_plot.png"
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"RMSD plot saved to {output_path}")

    return time, rmsd

## GFP

In [10]:
path = Path("./gfp-data/")

In [11]:
u = mda.Universe(path / "structure.pdb", path / "trajectory.xtc")

In [12]:
# compute rmsd for all atoms, backbone and calpha
compute_trajectory_rmsd(path / "structure.pdb", path / "trajectory.xtc")

RMSD plot saved to rmsd_plot.png


(array([0.000e+00, 1.000e+01, 2.000e+01, ..., 9.998e+04, 9.999e+04,
        1.000e+05], shape=(10001,)),
 array([2.69281219e-06, 7.54147502e-01, 8.86172865e-01, ...,
        3.57405994e+00, 4.08780492e+00, 3.54214474e+00], shape=(10001,)))

## Photosystem 2

In [15]:
path = Path("./ps-2-data/")

In [16]:
u = mda.Universe(path / "structure.pdb", path / "trajectory.xtc")

/Users/pablo/software/hackathon-local/.venv/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:350: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using universe.guess_TopologyAttrs(context='default', to_guess=['elements']).
  warnings.warn("Element information is missing, elements attribute "


In [17]:
# compute rmsd for all atoms, backbone and calpha
compute_trajectory_rmsd(path / "structure.pdb", path / "trajectory.xtc")

RMSD plot saved to rmsd_plot.png


(array([0.000e+00, 1.000e+01, 2.000e+01, ..., 9.998e+04, 9.999e+04,
        1.000e+05], shape=(10001,)),
 array([2.71500936e-06, 9.84501776e-01, 7.96082973e-01, ...,
        7.15606778e+00, 4.35904061e+00, 4.53432125e+00], shape=(10001,)))

## Cytochrome P450

In [18]:
path = Path("./c-p450-data/")

In [19]:
u = mda.Universe(path / "structure.pdb", path / "trajectory.xtc")

In [20]:
# compute rmsd for all atoms, backbone and calpha
compute_trajectory_rmsd(path / "structure.pdb", path / "trajectory.xtc")

RMSD plot saved to rmsd_plot.png


(array([0.000e+00, 1.000e+01, 2.000e+01, ..., 9.998e+04, 9.999e+04,
        1.000e+05], shape=(10001,)),
 array([4.15041084e-06, 9.26542245e-01, 1.05393071e+00, ...,
        2.32886573e+00, 2.24340516e+00, 2.25355164e+00], shape=(10001,)))